# Este cuaderno realiza upsampling de las muestras de entrenamiento para ML

In [1]:
## Liberías necesarias 
import os
import re
import socket 
import warnings
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE, ADASYN

## Configuración de rutas
if socket.gethostname()=='SRVCBECO01':
    os.chdir('D:\shared_data\Dropbox\Vías clinicas diabetes')
elif socket.gethostname() == 'HPJP2': 
    os.chdir(r'C:\Users\juanm\Dropbox\JP_files\UR\Vías clinicas diabetes')

raw_path = 'Datos org\\csv_rutas\\'
created_path = "Datos creados\\"
temp_path = 'temporales\\'


print('Directorio actual: '+ os.getcwd())

Directorio actual: C:\Users\juanm\Dropbox\JP_files\UR\Vías clinicas diabetes


### Funciones para realizar el upsampling

In [2]:
def db_upsample(tgt_cat, 
                vars_to_drop):
    train_paths = os.listdir(created_path+"ml_databases\\{}_cat\\train\\".format(tgt_cat))
    
    for train_path in train_paths:
        print('Upsampling {}'.format(train_path))
        ## Cargando el archivo 
        data_mat = pd.read_csv(created_path+"ml_databases\\{}_cat\\train\\{}".format(tgt_cat, train_path))
        org_len = len(data_mat)

        data_mat.dropna(inplace = True)

        if len(data_mat) != org_len:
            print("{} observations lost due to missing values.".format(org_len-len(data_mat)))

        ## Segmentando entre variables exógenas y endógenas
        exog_vars = data_mat.drop(vars_to_drop, axis = 1)
        endog_vars = data_mat.loc[:, 'tgt_label']

        ## Ejecutando el sobremuestro
        X_resampled, y_resampled = SMOTE(random_state = 123).fit_resample(exog_vars,
                                                                          endog_vars)

        ## Exportando la base sobremuestreada
        pd.concat([y_resampled,
                   X_resampled],
                  axis = 1).to_csv(temp_path+"ml_dbs\\{}_cat\\train_ryr_upsampled\\upsampled_{}".format(tgt_cat, train_path),
                                   index = False)

### Realizando el upsampling

In [3]:
## Parámetros estáticos
vars_to_drop = ['KeyAnonimo', 'year', 'base_label', 'tgt_label']


## Iterando entre las posibles cateogorizaciones de los outcomes
for tgt_cat in range(4, 5):
    db_upsample(tgt_cat = tgt_cat, 
               vars_to_drop = vars_to_drop)

Upsampling train_db_1_years_base_1.csv
Upsampling train_db_1_years_base_2.csv
Upsampling train_db_1_years_base_3.csv
Upsampling train_db_1_years_base_4.csv
Upsampling train_db_2_years_base_1.csv
Upsampling train_db_2_years_base_2.csv
Upsampling train_db_2_years_base_3.csv
Upsampling train_db_2_years_base_4.csv


### Verificando el balance

In [4]:
upsampled_paths = os.listdir(temp_path+"ml_dbs\\{}_cat\\train_ryr_upsampled\\".format(tgt_cat))
for path in upsampled_paths:
    print(pd.read_csv(temp_path+"ml_dbs\\{}_cat\\train_ryr_upsampled\\{}".format(tgt_cat, path)).loc[:, 'fuera_metas'].value_counts())

0    3486
1    3486
Name: fuera_metas, dtype: int64
1    2744
0    2744
Name: fuera_metas, dtype: int64
0    4787
1    4787
Name: fuera_metas, dtype: int64
1    4082
0    4082
Name: fuera_metas, dtype: int64
0    2990
1    2990
Name: fuera_metas, dtype: int64
1    2272
0    2272
Name: fuera_metas, dtype: int64
0    1752
1    1752
Name: fuera_metas, dtype: int64
1    1520
0    1520
Name: fuera_metas, dtype: int64
